<a href="https://colab.research.google.com/github/convergencelab/LSHT-HSLT-MODIS-Landsat-Fusion/blob/master/Super_Resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This project will look at taking landsat-MODIS pairs geographically allgined and preprocessed to the same dimensions. Cloud cover content remains below 10%. 




In [1]:
### imports ###
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pathlib
import shutil
import random
import functools
import tqdm

In [5]:
### Utilities for managing eurosat dataset ###


class data_manipulator:
    """
    data manipulator: takes dir with class name/imgs format and splits into
    train/test sets
    """
    def __init__(self, base_dir):
        self._base_dir = base_dir

    def train_test_split(self, validation=False, data_split=0.80):
        classes = os.listdir(self._base_dir)
        train_dir = self._base_dir + "/train_data"
        test_dir = self._base_dir + "/test_data"
        os.mkdir(train_dir)
        os.mkdir(test_dir)
        if validation:
            os.mkdir(self._base_dir + "/validation_data")
        for c in classes:
            train_class_dir = train_dir+"/"+c
            test_class_dir = test_dir + "/" + c
            os.mkdir(train_class_dir)
            os.mkdir(test_class_dir)
            if not validation:
                c_path = self._base_dir + "/" + c
                c_dir = os.listdir(c_path)
                #shuffle images so we take random ones each time for split
                random.shuffle(c_dir)
                print(c_dir)
                train = c_dir[0:int(data_split*len(c_dir))]
                test = c_dir[int(data_split*len(c_dir)):]
                #move all train data to train folder
                for train_data in train:
                    shutil.copy(c_path+ "/" + train_data, train_class_dir)
                # move all test data to test folder
                for test_data in test:
                    shutil.copy(c_path+ "/" + test_data, test_class_dir)
            """TODO: add validation"""
        print(classes)

def split_data(basedir, data_split=0.80):
    """
    quicker for calls in py console
    """
    manip = data_manipulator(basedir)
    manip.train_test_split(data_split=data_split)

def show_batch(image_batch):
    """
    visualization of batch
    :param image_batch: from ds_train typically
    :param label_batch: from ds_train typically
    :return: None`
    """
    fig, ax = plt.subplots(4, 3, figsize=(12,8))
    for n, b in enumerate(image_batch[0]):

        ax[n%4, n%3].imshow(b.numpy()
                            )
        plt.axis('off')

    plt.show()

# Train VGG-19 on Senteniel-2 dataset: EuroSat
VGG-19 is used in the architecture proposed in https://arxiv.org/abs/1609.04802:
  In novel perceptual loss function, content loss aspect transferred from pretrained VGG-19 using ImageNet dataset. For this project a Remotely sensed dataset may provide more useful features. 

**Step 1: Load in dataset**
The EuroSat dataset consists of 10 classes with in total 27,000 labeled and geo-referenced images. For this project we will train the VGG-19 deep convolutional net using this dataset, as it is likely to learn characteristics more similar to that of Landsat and MODIS.

ref: https://arxiv.org/pdf/1709.00029.pdf





In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

class loader(object):
    def __init__(self, base_dir, img_width=64, img_height=64):
        self._base_dir = pathlib.Path(base_dir)
        # toggling custom or all classes selection
        self._class_names = np.array([c.name for c in self._base_dir.glob('*')])
        #self._labels_as_bool = labels_as_bool
        self._image_count = len(list(self._base_dir.glob('*/*.jpg')))
        #batch_size as class feature is clunky
        #self._batch_size = batch_size
        #setting image width and height set here
        self._img_width = img_width
        self._img_height = img_height
        self._ds_size = 0

        # load 2 idx can be used to convert output to label
        self._current_class_load_idx_2_list = {}
        # current data set loaded in loader
        self._current_ds = {}



    def load_data(self, selected_classes=False, label_as_idx=True):
        """
        load data into memory
        :param dirs: if all, use every class in dir, else use selected files
        :return: None (void, loads data into loader object)
        """
        #self._ds = tf.data.Dataset.list_files(str(self._base_dir / '*/*'))
        if isinstance(selected_classes, list):
            to_load = selected_classes
        else:
            to_load = self._class_names
        """
        idx name to conversion
        """
        if label_as_idx:
            self._current_class_load_idx_2_list = {c: float(i) for i, c in enumerate(to_load)}
        else:
            # convert to bool
            self._current_class_load_idx_2_list = {c: c == self._class_names for c in to_load}
        """
        potentially block out to function
        """
        for c in to_load:
            # class files indexed using dict
            # dict: key is class label as index, val is shuffleDataset of list dir
            self._current_ds[self._current_class_load_idx_2_list[c]] = \
                tf.data.Dataset.list_files(str((self._base_dir/c/'*')))
            #print(str((self._base_dir / c / '*/*')))
        """
        convert loaded dirs to image tensors
        """
        self._process()


    def _decode_img(self, img):
        """
        take tf string and convert to img tensor
        :param img:
        :return:
        """
        img = tf.image.decode_jpeg(img, channels=3)
        # Use `convert_image_dtype` to convert to floats in the [0,1] range.
        img = tf.image.convert_image_dtype(img, tf.float32)
        # resize the image to the desired size.
        return tf.image.resize(img, [self._img_width, self._img_height])

    def _map_img(self, class_idx, img_file_path):
        """
        map image function
        :param class_idx:
        :return:
        """
        # iterate one class at a time
        label = tf.constant(np.array(class_idx).astype('float32').reshape((1)))
        # load file
        img = tf.io.read_file(img_file_path)
        # string tensor is converted to tf.image
        img = self._decode_img(img)
        return img, label



    def _process(self):
        """
        process the files into image, label tensors
        :return:
        """

        # intialize with first class
        keys = iter(self._current_ds.keys())
        init_idx = next(keys)
        this_map_func = functools.partial(self._map_img, class_idx=init_idx)
        _concat_ds = self._current_ds[init_idx].map(lambda x: this_map_func(img_file_path=x),
                                                    num_parallel_calls=AUTOTUNE)
        # iter through rest of keys
        for cur_class_idx in keys:
            this_map_func = functools.partial(self._map_img, class_idx=cur_class_idx)
            #convert from dir, and class, to img and class tensors
            _concat_ds = _concat_ds.concatenate(self._current_ds[cur_class_idx].map(lambda x: this_map_func(img_file_path=x),
                                                                        num_parallel_calls=AUTOTUNE))
        # assign concatonated ds to current_ds
        self._current_ds = _concat_ds
        # get size of ds to set to
        self._ds_size = tf.data.experimental.cardinality(self._current_ds).numpy()

    def get_ds_size(self):
        """
        currently train is entire dataset
        :return: size of training dataset
        """
        "TODO:"
        return self._ds_size

    def reset_load(self):
        """
        resets loaded data set and converter
        :return: None
        """
        ### reset ###
        self._current_class_load_idx_2_list = {}
        self._current_ds = {}

    def get_dataset(self):
        return self._current_ds


class training_data_loader(loader):

    def __init__(self, base_dir):
        super().__init__(base_dir)

    def prepare_for_training(self, batch_size=25, cache=True, shuffle_buffer_size=1000):
        """
        prepares dataset for training makes use of caching to speed up transfer
        cache is used so we only need to load the ds once and after that it will
        refer to cached data rather than reloading multiple instances into mem
        :param cache: bool toggle
        :param shuffle_buffer_size:
        :param batch_size: batch_size for prepared dataset
        :return: returns prepared ds
        """
        self._current_ds = self._current_ds.take(batch_size)
        if cache:
            if isinstance(cache, str):
                # if we pass a string to cache
                self._current_ds = self._current_ds.cache(cache)
            else:
                # filename not provided, stored in memory
                self._current_ds = self._current_ds.cache()
        # fills a buffer with buffer_size els
        # randomly samples els from this buffer
        self._current_ds = self._current_ds.shuffle(buffer_size=shuffle_buffer_size)

        # Repeat forever (indefinately) -> this duplicates the data N times
        self._current_ds = self._current_ds.repeat()

        # Seperates ds into batches of batch size, remainder is not dropped
        # may be a batch which has less els than others
        # N % D els in last batch
        self._current_ds = self._current_ds.batch(batch_size)

        # `prefetch` lets the dataset fetch batches in the background while the model
        # is training.
        self._current_ds = iter(self._current_ds.prefetch(buffer_size=AUTOTUNE))

    def get_train_batch(self):
        return next(self._current_ds)

class testing_data_loader(loader):
    def __init__(self, base_dir):
        super().__init__(base_dir)

    def prepare_for_testing(self, cache=True, shuffle_buffer_size=1000):
        """
        prepares dataset for training makes use of caching to speed up transfer
        cache is used so we only need to load the ds once and after that it will
        refer to cached data rather than reloading multiple instances into mem
        :param cache: bool toggle
        :param shuffle_buffer_size:
        :param batch_size: batch_size for prepared dataset
        :return: returns prepared ds
        """
        if cache:
            if isinstance(cache, str):
                # if we pass a string to cache
                self._current_ds = self._current_ds.cache(cache)
            else:
                # filename not provided, stored in memory
                self._current_ds = self._current_ds.cache()
        # fills a buffer with buffer_size els
        # randomly samples els from this buffer
        self._current_ds = self._current_ds.shuffle(buffer_size=shuffle_buffer_size)

        # Repeat forever (indefinately) -> this duplicates the data N times
        self._current_ds = self._current_ds.repeat()

        # Seperates ds into batches of batch size, remainder is not dropped
        # may be a batch which has less els than others
        # N % D els in last batch


        # `prefetch` lets the dataset fetch batches in the background while the model
        # is training.
        self._current_ds = self._current_ds.prefetch(buffer_size=AUTOTUNE)


    def get_test_batch(self, batch_size):
        self._current_ds = iter(self._current_ds.batch(batch_size))
        return next(self._current_ds)

**Load the Data**

In [8]:
### get data ###
"""
using eurosat dataset, this dataset uses the sentenial-2 collected satellite images
"""
euro_path = r"C:\Users\Noah Barrett\Desktop\School\Research 2020\data\EuroSat"

### Hyperparameters ###
batch_size = 1

### initalize loaders ###
train_data = training_data_loader(
    base_dir=os.path.join(euro_path, "train_data"))
test_data = testing_data_loader(
    base_dir=os.path.join(euro_path, "test_data"))
### load data ###
train_data.load_data()
test_data.load_data()

### prep train-data ###
train_data.prepare_for_training(batch_size=batch_size)
test_data.prepare_for_testing()

StopIteration: ignored

**Model Initialization, HyperParameters, and Training**


In [9]:
### initialize model ###
vgg = tf.keras.applications.VGG19(
                            include_top=True,
                            weights=None,
                            input_tensor=None,
                            input_shape=[224, 224, 3],
                            pooling=None,
                            classes=1000,
                            classifier_activation="softmax"
                        )

### loss function ###
"""
Use MSE loss:
  
    ref -> "https://towardsdatascience.com/loss-functions-based-on-feature-activation-and-style-loss-2f0b72fd32a9"
"""

m_loss = tf.keras.losses.MSE

### adam optimizer for SGD ###
optimizer = tf.keras.optimizers.Adam()

### intialize metrics ###
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_vgg-19_acc')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_vgg-19_acc')


### train step ###
@tf.function
def train_step(idx, sample, label):
  with tf.GradientTape() as tape:
    # preprocess for vgg-19
    sample = tf.image.resize(sample, (224, 224))
    sample = tf.keras.applications.vgg19.preprocess_input(sample)

    predictions = vgg(sample, training=True)
    # mean squared error in prediction
    loss = tf.keras.losses.MSE(label, predictions)

  # apply gradients
  gradients = tape.gradient(loss, vgg.trainable_variables)
  optimizer.apply_gradients(zip(gradients, vgg.trainable_variables))

  # update metrics
  train_loss(loss)
  train_accuracy(vgg, predictions)

### generator test step ###
@tf.function
def test_step(idx, sample, label):
  # preprocess for vgg-19
  sample = tf.image.resize(sample, (224, 224))
  sample = tf.keras.applications.vgg19.preprocess_input(sample)
  # feed test sample in
  predictions = vgg(sample, training=False)
  t_loss = tf.keras.losses.MSE(label, predictions)

  # update metrics
  test_loss(t_loss)
  test_accuracy(label, predictions)

### Weights Dir ###
if not os.path.isdir('./checkpoints'):
    os.mkdir('./checkpoints')

### TRAIN ###
EPOCHS = 1000
NUM_CHECKPOINTS_DIV = int(EPOCHS/4)
save_c = 1
### use CPU locally due to OOM error ###
with tf.device('/CPU:0'):
    for epoch in range(EPOCHS):
        # Reset the metrics at the start of the next epoch
        train_loss.reset_states()
        train_accuracy.reset_states()
        test_loss.reset_states()
        test_accuracy.reset_states()
        for idx in tqdm(range(train_data.get_ds_size() // batch_size)):
            # train step
            batch = train_data.get_train_batch()
            for sample, label in zip(batch[0], batch[1]):
                sample = np.array(sample)[np.newaxis, ...]
                label = np.array(label)[np.newaxis, ...]
                train_step(idx, sample, label)

            # test step
            batch = test_data.get_test_batch()
            for sample, label in zip(batch[0], batch[1]):
                sample = np.array(sample)[np.newaxis, ...]
                label = np.array(label)[np.newaxis, ...]

                test_step(idx, sample, label)

        ### save weights ###
        if not epoch % NUM_CHECKPOINTS_DIV:
            vgg.save_weights('./checkpoints/my_checkpoint_{}'.format(save_c))
            save_c += 1
        if not epoch % 100:
            ### outputs every 100 epochs so .out file from slurm is not huge. ###
            template = 'Training VGG-19:\nEpoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
            print(template.format(epoch + 1,
                                  train_loss.result(),
                                  train_accuracy.result() * 100,
                                  test_loss.result(),
                                  test_accuracy.result() * 100))


NameError: ignored

Differences between Landsat, MODIS and Senteniel-2:

**Landsat**: 
*Landsat 8 carries the OLI and TIRS sensors (this project focusses on OLI collected data*

*   Scene size: 170 km x 185 km (106 mi x 115 mi))
*   233 orbit cycle; covers the entire globe every 16 days (except for the highest polar latitudes)
*   Circles the Earth every 98.9 minutes


**MODIS**:
*The MOD09GA Version 6 product provides an estimate of the surface spectral reflectance of Terra Moderate Resolution Imaging Spectroradiometer (MODIS) Bands 1 through 7, corrected for atmospheric conditions such as gasses, aerosols, and Rayleigh scattering. *

*   500 meter (m) surface reflectance, observation, and quality bands are a set of ten 1 kilometer (km) observation bands and geolocation flags.

refs:
    

*   https://eos.com/landsat-8/
*   https://lpdaac.usgs.gov/products/mod09gav006/


In [ ]:
_CITATION = """
    @misc{helber2017eurosat,
    title={EuroSAT: A Novel Dataset and Deep Learning Benchmark for Land Use and Land Cover Classification},
    author={Patrick Helber and Benjamin Bischke and Andreas Dengel and Damian Borth},
    year={2017},
    eprint={1709.00029},
    archivePrefix={arXiv},
    primaryClass={cs.CV}
}"""